# Benchmarks - Juego Mancala

Analisis de metricas por configuracion y heuristica


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob
import numpy as np

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Librerias cargadas")


✓ Librerias cargadas


In [11]:
# Cargar todos los CSV de __pycache__
cache_dir = '__pycache__'
csv_files = [f for f in glob(os.path.join(cache_dir, '*')) if f.endswith('.csv')]

if not csv_files:
    print("⚠ No hay archivos de benchmark")
    df = None
else:
    print(f"✓ Encontrados {len(csv_files)} archivos")
    dfs = []
    for file in csv_files:
        try:
            df_temp = pd.read_csv(file)
            dfs.append(df_temp)
            print(f"  ✓ {os.path.basename(file)}: {len(df_temp)} partidas")
        except Exception as e:
            print(f"  ✗ Error en {file}: {e}")
    
    if dfs:
        df = pd.concat(dfs, ignore_index=True)
        print(f"\n✓ Total: {len(df)} partidas cargadas")
        print(f"\nColumnas disponibles:")
        print(df.columns.tolist())
        print(f"\nPrimeras filas:")
        display(df.head())
    else:
        df = None


✓ Encontrados 3 archivos
  ✓ benchmark_greedy_vs_minimax.csv: 2 partidas
  ✓ benchmark_random_vs_minimax.csv: 1 partidas
  ✓ benchmark_worst_vs_minimax.csv: 2 partidas

✓ Total: 5 partidas cargadas

Columnas disponibles:
['timestamp', 'modo_juego', 'jugador1_tipo', 'jugador2_tipo', 'time_limit', 'duracion_total', 'ganador', 'score_p1', 'score_p2', 'p1_nodos_expandidos', 'p1_profundidad_total', 'p1_movimientos', 'p2_nodos_expandidos', 'p2_profundidad_total', 'p2_movimientos', 'p1_profundidad_promedio', 'p2_profundidad_promedio']

Primeras filas:


,timestamp,modo_juego,jugador1_tipo,jugador2_tipo,time_limit,duracion_total,ganador,score_p1,score_p2,p1_nodos_expandidos,p1_profundidad_total,p1_movimientos,p2_nodos_expandidos,p2_profundidad_total,p2_movimientos,p1_profundidad_promedio,p2_profundidad_promedio
0,2025-10-28 21:06:58,4,IA (Greedy),IA (Minimax),2.0,129.218993,¡Jugador 2 Gana!,31,67,195,36.0,36.000000,2312498,347,64.000000,1.000000,5.421875
1,2025-10-28 21:13:10,4,IA (Greedy),IA (Minimax),2.0,129.054868,¡Jugador 2 Gana!,31,67,195,36.0,36.000000,2426381,349,64.000000,1.000000,5.453125
2,2025-10-28 20:47:33,5,IA (Random),IA (Minimax),2.0,178.611228,¡Jugador 2 Gana!,29,69,1738185,248.0,35.000000,35,35,35.000000,7.085714,1.000000
3,2025-10-28 21:35:37,6,IA (Worst),IA (Minimax),2.0,160.41920280456543,¡Jugador 2 Gana!,21,77,238,46.0,46.000000,2525168,441,79.000000,1.000000,5.582278
4,2025-10-28 22:53:01,6,IA (Worst),IA (Minimax),Facil,Jugador 2,77,21,238,2408623,1.0,5.556962,46,439,161.755561,NaN,NaN


## Analisis de Metricas por Configuracion


In [ ]:
# Mapear modos de juego
if df is not None:
    # Mapear modo_juego a nombre descriptivo
    modos = {
        1: 'Humano vs Humano',
        2: 'Humano vs IA',
        3: 'IA vs IA',
        4: 'Greedy vs Minimax',
        5: 'Random vs Minimax',
        6: 'Worst vs Minimax'
    }
    
    # Verificar si las columnas antiguas existen
    if 'modo_juego' in df.columns:
        df['modo_nombre'] = df['modo_juego'].map(modos)
    else:
        df['modo_nombre'] = 'Desconocido'
    
    print("Columnas disponibles en el DataFrame:")
    print(df.columns.tolist())
    print("\nDistribucion por modo:")
    if 'modo_nombre' in df.columns:
        print(df['modo_nombre'].value_counts())
    print("\nDistribucion por dificultad:")
    if 'dificultad' in df.columns:
        print(df['dificultad'].value_counts())
    else:
        print("Columna 'dificultad' no encontrada")


Distribucion por modo:
modo_nombre
Greedy vs Minimax    2
Worst vs Minimax     2
Random vs Minimax    1
Name: count, dtype: int64

Distribucion por dificultad:


KeyError: 'dificultad'

In [ ]:
# Resumen de metricas principales
if df is not None:
    print("RESUMEN DE METRICAS PRINCIPALES:")
    print("=" * 60)
    
    # Usar nombres de columnas que puedan existir (antiguos o nuevos)
    col_ganador = 'ganador_del_juego' if 'ganador_del_juego' in df.columns else 'ganador'
    col_puntos_ganador = 'cantidad_de_puntos_obtenidos_ganador' if 'cantidad_de_puntos_obtenidos_ganador' in df.columns else 'puntos_ganador'
    col_nodos_j1 = 'cantidad_de_nodos_expandidos_j1' if 'cantidad_de_nodos_expandidos_j1' in df.columns else 'nodos_expandidos_j1'
    col_nodos_j2 = 'cantidad_de_nodos_expandidos_j2' if 'cantidad_de_nodos_expandidos_j2' in df.columns else 'nodos_expandidos_j2'
    col_tiempo = 'tiempo_de_ejecucion' if 'tiempo_de_ejecucion' in df.columns else 'tiempo_ejecucion'
    col_prof_prom_j1 = 'promedio_de_profundidad_j1' if 'promedio_de_profundidad_j1' in df.columns else 'profundidad_promedio_j1'
    col_prof_prom_j2 = 'promedio_de_profundidad_j2' if 'promedio_de_profundidad_j2' in df.columns else 'profundidad_promedio_j2'
    col_prof_total_j1 = 'profundidad_total_del_arbol_de_busqueda_j1' if 'profundidad_total_del_arbol_de_busqueda_j1' in df.columns else 'profundidad_total_j1'
    col_prof_total_j2 = 'profundidad_total_del_arbol_de_busqueda_j2' if 'profundidad_total_del_arbol_de_busqueda_j2' in df.columns else 'profundidad_total_j2'
    
    print(f"\n1. Ganador del Juego:")
    if col_ganador in df.columns:
        print(df[col_ganador].value_counts())
    else:
        print("Columna no encontrada")
    
    print(f"\n2. Cantidad de puntos obtenidos (Ganador):")
    if col_puntos_ganador in df.columns:
        print(f"   Min: {df[col_puntos_ganador].min()}")
        print(f"   Max: {df[col_puntos_ganador].max()}")
        print(f"   Promedio: {df[col_puntos_ganador].mean():.2f}")
    else:
        print("Columna no encontrada")
    
    print(f"\n3. Cantidad de nodos expandidos:")
    if col_nodos_j1 in df.columns and col_nodos_j2 in df.columns:
        print(f"   Jugador 1 - Min: {df[col_nodos_j1].min()}, Max: {df[col_nodos_j1].max()}")
        print(f"   Jugador 2 - Min: {df[col_nodos_j2].min()}, Max: {df[col_nodos_j2].max()}")
    else:
        print("Columnas no encontradas")
    
    print(f"\n4. Tiempo de ejecucion:")
    if col_tiempo in df.columns:
        print(f"   Min: {df[col_tiempo].min():.2f}s")
        print(f"   Max: {df[col_tiempo].max():.2f}s")
        print(f"   Promedio: {df[col_tiempo].mean():.2f}s")
    else:
        print("Columna no encontrada")
    
    print(f"\n5. Promedio de Profundidad:")
    if col_prof_prom_j1 in df.columns and col_prof_prom_j2 in df.columns:
        print(f"   Jugador 1 - Promedio: {df[col_prof_prom_j1].mean():.2f}")
        print(f"   Jugador 2 - Promedio: {df[col_prof_prom_j2].mean():.2f}")
    else:
        print("Columnas no encontradas")
    
    print(f"\n6. Profundidad total del arbol de busqueda:")
    if col_prof_total_j1 in df.columns and col_prof_total_j2 in df.columns:
        print(f"   Jugador 1 - Total: {df[col_prof_total_j1].sum()}")
        print(f"   Jugador 2 - Total: {df[col_prof_total_j2].sum()}")
    else:
        print("Columnas no encontradas")


RESUMEN DE METRICAS PRINCIPALES:

1. Ganador del Juego:
ganador
¡Jugador 2 Gana!    4
77                  1
Name: count, dtype: int64

2. Cantidad de puntos obtenidos (Ganador):


KeyError: 'puntos_ganador'

## Tabla Resumen por Configuracion


In [ ]:
# Crear tabla resumen por configuracion y dificultad
if df is not None:
    # Mapear nombres de columnas antiguas y nuevas
    col_ganador = 'ganador_del_juego' if 'ganador_del_juego' in df.columns else 'ganador'
    col_puntos_ganador = 'cantidad_de_puntos_obtenidos_ganador' if 'cantidad_de_puntos_obtenidos_ganador' in df.columns else 'puntos_ganador'
    col_nodos_j1 = 'cantidad_de_nodos_expandidos_j1' if 'cantidad_de_nodos_expandidos_j1' in df.columns else 'nodos_expandidos_j1'
    col_nodos_j2 = 'cantidad_de_nodos_expandidos_j2' if 'cantidad_de_nodos_expandidos_j2' in df.columns else 'nodos_expandidos_j2'
    col_tiempo = 'tiempo_de_ejecucion' if 'tiempo_de_ejecucion' in df.columns else 'tiempo_ejecucion'
    col_prof_prom_j1 = 'promedio_de_profundidad_j1' if 'promedio_de_profundidad_j1' in df.columns else 'profundidad_promedio_j1'
    col_prof_prom_j2 = 'promedio_de_profundidad_j2' if 'promedio_de_profundidad_j2' in df.columns else 'profundidad_promedio_j2'
    col_prof_total_j1 = 'profundidad_total_del_arbol_de_busqueda_j1' if 'profundidad_total_del_arbol_de_busqueda_j1' in df.columns else 'profundidad_total_j1'
    col_prof_total_j2 = 'profundidad_total_del_arbol_de_busqueda_j2' if 'profundidad_total_del_arbol_de_busqueda_j2' in df.columns else 'profundidad_total_j2'
    
    # Agrupar por modo_nombre y dificultad si existen
    group_cols = []
    if 'modo_nombre' in df.columns:
        group_cols.append('modo_nombre')
    if 'dificultad' in df.columns:
        group_cols.append('dificultad')
    
    if group_cols:
        agg_dict = {}
        if col_ganador in df.columns:
            agg_dict[col_ganador] = lambda x: x.value_counts().to_dict()
        if col_puntos_ganador in df.columns:
            agg_dict[col_puntos_ganador] = ['min', 'max', 'mean']
        if col_nodos_j1 in df.columns and col_nodos_j2 in df.columns:
            agg_dict[col_nodos_j1] = ['sum', 'mean']
            agg_dict[col_nodos_j2] = ['sum', 'mean']
        if col_tiempo in df.columns:
            agg_dict[col_tiempo] = ['min', 'max', 'mean']
        if col_prof_prom_j1 in df.columns and col_prof_prom_j2 in df.columns:
            agg_dict[col_prof_prom_j1] = 'mean'
            agg_dict[col_prof_prom_j2] = 'mean'
        if col_prof_total_j1 in df.columns and col_prof_total_j2 in df.columns:
            agg_dict[col_prof_total_j1] = 'sum'
            agg_dict[col_prof_total_j2] = 'sum'
        
        resumen = df.groupby(group_cols).agg(agg_dict).round(2)
        
        print("TABLA RESUMEN POR CONFIGURACION Y DIFICULTAD")
        print("=" * 100)
        display(resumen)
    else:
        print("No se pueden agrupar por modo_nombre o dificultad")


KeyError: 'dificultad'

## Tabla segun Estructura Solicitada


In [ ]:
# Crear tabla segun la estructura solicitada
# Configuraciones -> Ganador -> Puntos -> Nodos -> Tiempo -> Prof Prom -> Prof Total
if df is not None:
    # Filtrar solo por configuraciones IA vs IA (modo 4, 5, 6)
    df_ia = df[df['modo_juego'].isin([4, 5, 6])].copy() if 'modo_juego' in df.columns else df.copy()
    
    # Mapear modos
    config_map = {
        4: '1ra configuracion de pesos (Greedy vs Minimax)',
        5: '2da configuracion de pesos (Random vs Minimax)',
        6: '3ra configuracion de pesos (Worst vs Minimax)'
    }
    
    if 'modo_juego' in df.columns:
        df_ia['Configuraciones'] = df_ia['modo_juego'].map(config_map)
    else:
        df_ia['Configuraciones'] = 'Desconocido'
    
    # Renombrar columnas para la tabla final
    col_map = {
        'ganador_del_juego': 'Ganador del Juego',
        'cantidad_de_puntos_obtenidos_ganador': 'Cantidad de puntos obtenidos',
        'cantidad_de_nodos_expandidos_j1': 'Cantidad de nodos expandidos J1',
        'cantidad_de_nodos_expandidos_j2': 'Cantidad de nodos expandidos J2',
        'tiempo_de_ejecucion': 'Tiempo de ejecucion',
        'promedio_de_profundidad_j1': 'Promedio de Profundidad J1',
        'promedio_de_profundidad_j2': 'Promedio de Profundidad J2',
        'profundidad_total_del_arbol_de_busqueda_j1': 'Profundidad total del arbol de busqueda J1',
        'profundidad_total_del_arbol_de_busqueda_j2': 'Profundidad total del arbol de busqueda J2',
        'ganador': 'Ganador del Juego',
        'puntos_ganador': 'Cantidad de puntos obtenidos',
        'nodos_expandidos_j1': 'Cantidad de nodos expandidos J1',
        'nodos_expandidos_j2': 'Cantidad de nodos expandidos J2',
        'tiempo_ejecucion': 'Tiempo de ejecucion',
        'profundidad_promedio_j1': 'Promedio de Profundidad J1',
        'profundidad_promedio_j2': 'Promedio de Profundidad J2',
        'profundidad_total_j1': 'Profundidad total del arbol de busqueda J1',
        'profundidad_total_j2': 'Profundidad total del arbol de busqueda J2'
    }
    
    # Crear DataFrame con estructura solicitada
    tabla_final = df_ia.rename(columns=col_map)
    
    # Mostrar solo las columnas relevantes
    cols_mostrar = ['Configuraciones', 'dificultad'] + [c for c in col_map.values() if c in tabla_final.columns]
    cols_available = [c for c in cols_mostrar if c in tabla_final.columns]
    
    if cols_available:
        print("TABLA DE METRICAS POR CONFIGURACION")
        print("=" * 120)
        display(tabla_final[cols_available])
    else:
        print("No se encontraron las columnas necesarias")
        display(tabla_final)
